# Google Analytics Customer Revenue Prediction

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

Define a function to load the CSV files

In [ ]:
%%time
# https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields/notebook
def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    json = __import__('json')
    json_normalize = pd.io.json.json_normalize
    df = pd.read_csv(
        csv_path,
        converters={ column: json.loads for column in JSON_COLUMNS },
        dtype={ 'fullVisitorId': 'str', 'visitId': 'str' },
        nrows=nrows
    )
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f'{column}.{subcolumn}' for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f'Loaded {csv_path}. Shape: {df.shape}')
    return df

# df_train = load_df()
# df_test = load_df('../input/test.csv')

In [ ]:
# %%time
# df_train.to_csv('train-flattened.csv', index=False)
# df_test.to_csv('test-flattened.csv', index=False)

In [2]:
df_train = pd.read_csv('../input/ga-revenue-prediction/train-flattened.csv', dtype={ 'fullVisitorId': str, 'visitId': str, 'trafficSource.campaignCode': str },)
df_test = pd.read_csv('../input/ga-revenue-prediction/test-flattened.csv', dtype={ 'fullVisitorId': str, 'visitId': str },)

## Target variable
`totals.transactionRevenue`
1. Convert to float, fill NA with 0
2. Only 11515 records resulted in revenue, the remaining 892138 records had zero revenue.
3. Of these, only 9996 visitors (out of 714167 unique visitors) had non-zero revenue. This translates to: only 1.4% of visitors bought something.

In [3]:
df_train['totals.transactionRevenue'] = df_train['totals.transactionRevenue'].astype(float)
df_train['totals.transactionRevenue'].notnull().sum()
np.sum(df_train.groupby('fullVisitorId')['totals.transactionRevenue'].sum() > 0)

9996

## Predictors
1. There are 54 predictos in the train set and 52 in the test set.
2. Remove predictors not present in the test set
3. Remove predictors where values are the same for all observations



In [4]:
columns = set(df_train.columns).intersection((set(df_test.columns))) # columns present in both train and test
print('Remove columns:', end=' ')
for i in df_train.columns[df_train.nunique(dropna=False) == 1]:
    print(i, end=', ')
    columns.remove(i)

Remove columns: socialEngagementType, device.browserSize, device.browserVersion, device.flashVersion, device.language, device.mobileDeviceBranding, device.mobileDeviceInfo, device.mobileDeviceMarketingName, device.mobileDeviceModel, device.mobileInputSelector, device.operatingSystemVersion, device.screenColors, device.screenResolution, geoNetwork.cityId, geoNetwork.latitude, geoNetwork.longitude, geoNetwork.networkLocation, totals.visits, trafficSource.adwordsClickInfo.criteriaParameters, 

In [ ]:
def make_button(var, title = '', df = df_train, target_var = 'totals.transactionRevenue'):
    # create data for the updatemenus used by plotly
    # agg data from var ~ target_var into size, count, mean, median
    # return dict()
    tmp = df.groupby(var)[target_var].agg(['size', 'count', 'mean', 'median'])
    tmp = tmp.sort_values('size', ascending=False)[:10][::-1]
    tmp = {
        'x': [tmp['size'].values, tmp['count'].values, tmp['mean'].values, tmp['median'].values],
        'y': [[str(i) for i in tmp.index.tolist()]] * 4, # str(i) to convert all to string, because some of the indexes are True, False
    }
    title = title or var
    return dict(args=[tmp, { 'title': title }], label=title, method='update') 

# plotting
## data
tmp = make_button('device.deviceCategory', 'Device Category')
x = tmp['args'][0]['x']
y = tmp['args'][0]['y'][0]

## trace
traces = [None] * 4
traces[0] = (go.Bar(x=x[0], y=y, orientation='h'))
traces[1] = (go.Bar(x=x[1], y=y, orientation='h'))
traces[2] = (go.Bar(x=x[2], y=y, orientation='h', name='Mean'))
traces[3] = (go.Bar(x=x[3], y=y, orientation='h', name='Median'))

## fig, subplot
fig = __import__('plotly').tools.make_subplots(1, 3, subplot_titles=['Number of record', 'Number of record with revenue', 'Mean & Median'])
for i in range(3): fig.append_trace(traces[i], 1, i + 1)
fig.append_trace(traces[3], 1, 3)

## fig, layout
fig.layout.title = tmp['args'][1]['title']
fig.layout.showlegend = False
fig.layout.updatemenus = list([
    dict(
        buttons=[make_button(i) for i in [
            'device.deviceCategory', 'device.operatingSystem', 'device.browser', 'device.isMobile',
            'geoNetwork.continent', 'geoNetwork.subContinent', 'geoNetwork.country', 'geoNetwork.metro', 'geoNetwork.city', 'geoNetwork.region', 'geoNetwork.networkDomain',
        ]],
        direction = 'down',
        showactive = True,
        x = 0,
        xanchor = 'left',
        y = 1.25,
        yanchor = 'top' 
    ),
])

## now plot
py.iplot(fig)